In [19]:
"""
Hall & Jones (1999) Replication — Analysis Script
==================================================
Produces:
  - Table I  : Levels accounting decomposition
  - Table II : IV regression (OLS + 2SLS, robust SEs)
  - Figure I : Y/L vs social infrastructure scatter
  - Figure II: TFP vs social infrastructure scatter

Two samples throughout:
  (A) Complete cases only  — 93 countries, no imputation
  (B) Imputed sample       — ~127 countries, missing openness/governance
                             imputed via OLS on instruments (Hall & Jones method)

Run from any directory. Reads hj_master.csv from outputs folder.
"""

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ── paths ──────────────────────────────────────────────────────────────────
MASTER  = 'C:\\Users\\Adams\\OneDrive\\DE & E Research\\outputs\\merged.csv'
OUT_DIR = 'C:\\Users\\Adams\\OneDrive\\DE & E Research\\outputs\\'

ALPHA = 1/3   # capital share (Hall & Jones assumption)

In [20]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 1 — LOAD AND PREPARE DATA
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 65)
print("  Hall & Jones (1999) Replication")
print("=" * 65)

df = pd.read_csv(MASTER, encoding='latin1')
print(f"\nLoaded {len(df)} countries from master dataset")

# ── 1a. Mining correction ──────────────────────────────────────────────────
# Hall & Jones subtract mining value added from GDP before the accounting.
# If mining share is missing, assume 0 (conservative — documented deviation).
df['mining_va'] = df['mining_va'].fillna(0.0)

# GDP net of mining (fraction of total GDP remaining after removing mining VA)
df['gdp_nonmining_share'] = 1.0 - df['mining_va']

# Adjust Y/L for mining: multiply by non-mining share
# (If mining is 30% of GDP, we want Y/L to reflect only the non-mining economy)
df['yl_adj'] = df['yl'] * df['gdp_nonmining_share']

# ── 1b. Core variables for accounting ────────────────────────────────────
# log output per worker (adjusted for mining)
df['log_yl']  = np.log(df['yl_adj'])

# Capital term: (K/Y)^(alpha/(1-alpha))
# With alpha=1/3: exponent = (1/3)/(2/3) = 0.5  → square root of K/Y
df['cap_term'] = df['ky_ratio'] ** (ALPHA / (1 - ALPHA))

# Human capital h already computed (Mincerian index)
df['hc_term'] = df['hc']

# TFP residual A = (Y/L) / [cap_term × hc_term]
# i.e. Y/L = cap_term × hc_term × A  →  A = Y/L / (cap_term × hc_term)
df['tfp'] = df['yl_adj'] / (df['cap_term'] * df['hc_term'])

# log versions for scatter plots
df['log_si']  = np.log(df['social_infra'].clip(lower=1e-6))
df['log_tfp'] = np.log(df['tfp'].clip(lower=1e-6))

# ── 1c. Instrument list ──────────────────────────────────────────────────
INSTRUMENTS = ['distancefromeq', 'fr_trade', 'english_frac', 'we_lang_frac']

# ── 1d. Complete-case sample (Sample A) ──────────────────────────────────
core_vars = ['yl_adj', 'ky_ratio', 'hc', 'social_infra',
             'distancefromeq', 'fr_trade', 'english_frac', 'we_lang_frac']
sA = df.dropna(subset=core_vars).copy().reset_index(drop=True)
print(f"Sample A (complete cases):  {len(sA)} countries")


  Hall & Jones (1999) Replication

Loaded 183 countries from master dataset
Sample A (complete cases):  89 countries


In [21]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 2 — IMPUTATION (Hall & Jones method)
# ═══════════════════════════════════════════════════════════════════════════
print("\n--- Imputing missing social infrastructure ---")

# Hall & Jones impute missing governance/openness by regressing each component
# on the four instruments + quadratic distance from equator.
# We apply the same logic to social_infra directly (since we already
# combined governance and openness into one index).

def ols_impute(df_full, target_col, instrument_cols):
    """
    Regress target_col on instrument_cols (+ dist^2) using complete cases,
    then predict for all rows. Returns a new Series with imputed values.
    """
    # Add quadratic distance term
    regressors = instrument_cols + ['distancefromeq_sq']
    df_full = df_full.copy()
    df_full['distancefromeq_sq'] = df_full['distancefromeq'] ** 2

    # Training set: rows where both target and all regressors are observed
    train = df_full.dropna(subset=[target_col] + regressors)
    
    X_train = np.column_stack([np.ones(len(train))] +
                               [train[c].values for c in regressors])
    y_train = train[target_col].values

    # OLS: beta = (X'X)^{-1} X'y
    beta = np.linalg.lstsq(X_train, y_train, rcond=None)[0]

    # Predict for all rows that have complete instruments
    pred_col = target_col + '_imputed'
    df_full[pred_col] = np.nan
    has_inst = df_full.dropna(subset=regressors)
    X_pred = np.column_stack([np.ones(len(has_inst))] +
                               [has_inst[c].values for c in regressors])
    preds = X_pred @ beta
    # Clip predictions to [0, 1] (social infrastructure is bounded)
    preds = np.clip(preds, 0.0, 1.0)
    df_full.loc[has_inst.index, pred_col] = preds

    return df_full[pred_col], beta

# Impute social infrastructure
si_imputed, beta_si = ols_impute(df, 'social_infra', INSTRUMENTS)

# Build imputed series: use observed value if available, else imputed
df['social_infra_imp'] = df['social_infra'].combine_first(si_imputed)

n_imputed = df['social_infra'].isna().sum() - df['social_infra_imp'].isna().sum()
print(f"  Imputed social infrastructure for {n_imputed} additional countries")
print(f"  Imputation regression R²: {stats.pearsonr(df.dropna(subset=['social_infra','social_infra_imp'])['social_infra'], df.dropna(subset=['social_infra','social_infra_imp'])['social_infra_imp'])[0]**2:.3f}")

# ── Sample B: imputed sample ──────────────────────────────────────────────
core_vars_B = ['yl_adj', 'ky_ratio', 'hc', 'social_infra_imp',
               'distancefromeq', 'fr_trade', 'english_frac', 'we_lang_frac']
sB = df.dropna(subset=core_vars_B).copy().reset_index(drop=True)
sB['social_infra'] = sB['social_infra_imp']
sB['log_si']  = np.log(sB['social_infra'].clip(lower=1e-6))
sB['tfp']     = sB['yl_adj'] / (sB['cap_term'] * sB['hc_term'])
sB['log_tfp'] = np.log(sB['tfp'].clip(lower=1e-6))
sB['log_yl']  = np.log(sB['yl_adj'])
print(f"Sample B (with imputation): {len(sB)} countries")




--- Imputing missing social infrastructure ---
  Imputed social infrastructure for 38 additional countries
  Imputation regression R²: 1.000
Sample B (with imputation): 113 countries


In [22]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 3 — ECONOMETRICS TOOLKIT (OLS and 2SLS from scratch)
# ═══════════════════════════════════════════════════════════════════════════

def ols(y, X):
    """
    OLS regression.
    Returns dict: beta, residuals, fitted, n, k, HC1 robust covariance matrix.
    X should include a constant column.
    """
    n, k = X.shape
    XX_inv = np.linalg.inv(X.T @ X)
    beta   = XX_inv @ X.T @ y
    yhat   = X @ beta
    e      = y - yhat

    # HC1 (heteroskedasticity-robust) covariance: (X'X)^{-1} X' diag(e²) X (X'X)^{-1} * n/(n-k)
    meat   = X.T @ np.diag(e**2) @ X
    V_hc1  = (n / (n - k)) * XX_inv @ meat @ XX_inv

    ss_res = e @ e
    ss_tot = ((y - y.mean()) ** 2).sum()
    r2     = 1 - ss_res / ss_tot

    return {'beta': beta, 'e': e, 'yhat': yhat,
            'n': n, 'k': k, 'V': V_hc1, 'r2': r2}


def tsls(y, X_endog, X_exog, Z):
    """
    Two-Stage Least Squares (2SLS / IV).

    y       : outcome (n,)
    X_endog : endogenous regressors (n, p) — social infrastructure here
    X_exog  : included exogenous regressors (n, q) — just [1] (constant) here
    Z       : excluded instruments (n, m) — the four geographic/language vars

    Returns same dict as ols() plus first-stage F-stat and overid test.
    """
    n = len(y)

    # Full instrument matrix for first stage
    Z_full = np.column_stack([X_exog, Z])   # [const, instruments]
    X_full = np.column_stack([X_exog, X_endog])  # [const, S]

    # First stage: regress S on [const, instruments]
    fs = ols(X_endog.ravel(), Z_full)
    S_hat = fs['yhat'].reshape(-1, 1)

    # First-stage F-statistic (robust)
    # Test that all instrument coefficients are jointly zero
    # Positions of instrument coefficients: columns 1 onwards in Z_full
    n_inst = Z.shape[1]
    R = np.zeros((n_inst, Z_full.shape[1]))
    for i in range(n_inst):
        R[i, i + X_exog.shape[1]] = 1.0
    r = np.zeros(n_inst)
    Rb_r = R @ fs['beta'] - r
    V_fs  = fs['V']
    F_stat = (Rb_r @ np.linalg.inv(R @ V_fs @ R.T) @ Rb_r) / n_inst

    # Second stage: regress y on [const, S_hat]
    X2 = np.column_stack([X_exog, S_hat])
    ss = ols(y, X2)

    # Correct 2SLS standard errors:
    # Use actual residuals (y - X_full @ beta_2sls), not second-stage residuals
    beta_2sls = ss['beta']
    e_2sls    = y - X_full @ beta_2sls

    # HC1 robust covariance using actual residuals
    XX_inv_2 = np.linalg.inv(X2.T @ X2)
    meat_2   = X2.T @ np.diag(e_2sls**2) @ X2
    k2       = X2.shape[1]
    V_2sls   = (n / (n - k2)) * XX_inv_2 @ meat_2 @ XX_inv_2

    ss_res  = e_2sls @ e_2sls
    ss_tot  = ((y - y.mean())**2).sum()
    r2_2sls = 1 - ss_res / ss_tot

    # Sargan overidentification test (if more instruments than endogenous vars)
    # Regress 2SLS residuals on full instrument matrix; n*R² ~ chi²(m-p)
    overid_p = np.nan
    if n_inst > 1:
        sar = ols(e_2sls, Z_full)
        sargan_stat = n * sar['r2']
        df_sar = n_inst - X_endog.shape[1] if X_endog.ndim > 1 else n_inst - 1
        overid_p = 1 - stats.chi2.cdf(sargan_stat, df=df_sar)

    return {'beta': beta_2sls, 'e': e_2sls, 'yhat': ss['yhat'],
            'n': n, 'k': k2, 'V': V_2sls, 'r2': r2_2sls,
            'fs_beta': fs['beta'], 'fs_V': V_fs, 'F_first': F_stat,
            'overid_p': overid_p}


def fmt_result(res, labels):
    """Print a formatted regression result block."""
    beta = res['beta']
    V    = res['V']
    se   = np.sqrt(np.diag(V))
    t    = beta / se
    p    = 2 * (1 - stats.t.cdf(np.abs(t), df=res['n'] - res['k']))

    lines = []
    for i, lbl in enumerate(labels):
        stars = '***' if p[i] < 0.01 else ('**' if p[i] < 0.05 else
                ('*' if p[i] < 0.10 else ''))
        lines.append(f"  {lbl:<22} {beta[i]:>10.4f}  ({se[i]:.4f}) {stars}")
    lines.append(f"  {'N':<22} {res['n']:>10}")
    lines.append(f"  {'R²':<22} {res['r2']:>10.4f}")
    if 'F_first' in res:
        lines.append(f"  {'First-stage F':<22} {res['F_first']:>10.2f}")
    if 'overid_p' in res and not np.isnan(res['overid_p']):
        lines.append(f"  {'Overid test p-val':<22} {res['overid_p']:>10.3f}")
    return '\n'.join(lines)



In [23]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 4 — TABLE I: LEVELS ACCOUNTING
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 65)
print("  TABLE I — Levels Accounting Decomposition")
print("=" * 65)
print("  Methodology: Y/L = (K/Y)^(α/(1-α)) × h × A,  α = 1/3")
print("  All values relative to the United States = 1.00")
print()

def levels_accounting(sample, label):
    s = sample.copy()

    # Normalise everything to US = 1
    usa = s[s['iso3'] == 'USA'].iloc[0]
    s['rel_yl']  = s['yl_adj']   / usa['yl_adj']
    s['rel_cap'] = s['cap_term'] / usa['cap_term']
    s['rel_hc']  = s['hc_term']  / usa['hc_term']
    s['rel_tfp'] = s['tfp']      / usa['tfp']

    # Variance decomposition (in logs, following Hall & Jones eq. 6)
    # var(log Y/L) = cov(log Y/L, log cap) + cov(log Y/L, log h) + cov(log Y/L, log A)
    log_yl  = np.log(s['rel_yl'].clip(1e-6))
    log_cap = np.log(s['rel_cap'].clip(1e-6))
    log_hc  = np.log(s['rel_hc'].clip(1e-6))
    log_tfp_r = np.log(s['rel_tfp'].clip(1e-6))

    var_yl  = np.var(log_yl)
    cov_cap = np.cov(log_yl, log_cap)[0, 1]
    cov_hc  = np.cov(log_yl, log_hc)[0, 1]
    cov_tfp = np.cov(log_yl, log_tfp_r)[0, 1]

    share_cap = cov_cap / var_yl
    share_hc  = cov_hc  / var_yl
    share_tfp = cov_tfp / var_yl

    # Range: ratio of 90th to 10th percentile
    p90_p10 = lambda x: np.percentile(x, 90) / np.percentile(x, 10)

    print(f"\n  [{label}]  N = {len(s)}")
    print(f"\n  {'Country':<25} {'Y/L':>8} {'Cap term':>10} {'h':>8} {'TFP (A)':>10}  {'S':>8}")
    print(f"  {'-'*70}")

    # Show 15 representative countries (sorted by Y/L)
    show = s.sort_values('rel_yl', ascending=False)
    display_isos = ['USA','CHE','CAN','AUS','GBR','JPN','FRA','DEU',
                    'BRA','COL','THA','CHN','IND','KEN','NGA','NER']
    show_rows = show[show['iso3'].isin(display_isos)]
    for _, row in show_rows.iterrows():
        si_val = f"{row['social_infra']:.3f}" if not pd.isna(row['social_infra']) else '  NA '
        print(f"  {row['country']:<25} {row['rel_yl']:>8.3f} {row['rel_cap']:>10.3f} "
              f"{row['rel_hc']:>8.3f} {row['rel_tfp']:>10.3f}  {si_val:>8}")

    print(f"\n  90th/10th percentile ratios:")
    print(f"    Y/L:      {p90_p10(s['rel_yl']):.1f}x")
    print(f"    Cap term: {p90_p10(s['rel_cap']):.1f}x")
    print(f"    h:        {p90_p10(s['rel_hc']):.1f}x")
    print(f"    TFP (A):  {p90_p10(s['rel_tfp']):.1f}x")

    print(f"\n  Variance decomposition (fraction of var(log Y/L) explained):")
    print(f"    Capital term: {share_cap:.3f}")
    print(f"    Education h:  {share_hc:.3f}")
    print(f"    TFP (A):      {share_tfp:.3f}")
    print(f"    Sum:          {share_cap + share_hc + share_tfp:.3f}")

    return s  # return enriched sample with rel_ columns

sA_enriched = levels_accounting(sA, "Sample A — complete cases, no imputation")
sB_enriched = levels_accounting(sB, "Sample B — with imputation")




  TABLE I — Levels Accounting Decomposition
  Methodology: Y/L = (K/Y)^(α/(1-α)) × h × A,  α = 1/3
  All values relative to the United States = 1.00


  [Sample A — complete cases, no imputation]  N = 89

  Country                        Y/L   Cap term        h    TFP (A)         S
  ----------------------------------------------------------------------
  United States                1.000      1.000    1.000      1.000     0.917
  Switzerland                  0.894      1.290    0.887      0.782     0.947
  Canada                       0.849      0.970    0.855      1.024     0.907
  France                       0.813      1.065    0.656      1.165     0.793
  Australia                    0.802      1.053    0.927      0.821     0.759
  Japan                        0.639      1.073    0.822      0.725     0.738
  Colombia                     0.290      0.929    0.546      0.572     0.298
  Brazil                       0.204      0.846    0.477      0.506     0.278
  Thailand         

In [24]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 5 — TABLE II: IV REGRESSIONS
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 65)
print("  TABLE II — OLS and 2SLS Regressions")
print("  Dependent variable: log(Y/L)")
print("  Key regressor: Social infrastructure S ∈ [0,1]")
print("  Robust (HC1) standard errors in parentheses")
print("=" * 65)

def run_regressions(sample, label):
    s = sample.copy().dropna(
        subset=['log_yl','social_infra'] + INSTRUMENTS
    )
    n = len(s)

    y  = s['log_yl'].values
    S  = s['social_infra'].values.reshape(-1, 1)
    X1 = np.ones((n, 1))                           # just constant
    Z  = s[INSTRUMENTS].values

    # ── OLS ──
    X_ols  = np.column_stack([X1, S])
    res_ols = ols(y, X_ols)

    # ── 2SLS (all four instruments) ──
    res_iv = tsls(y, S, X1, Z)

    print(f"\n  [{label}]  N = {n}")
    print(f"\n  OLS:")
    print(fmt_result(res_ols, ['Constant', 'Social infra (S)']))
    print(f"\n  2SLS (instruments: dist_eq, FR trade, English frac, WE lang frac):")
    print(fmt_result(res_iv, ['Constant', 'Social infra (S)']))

    # ── 2SLS with subsets of instruments (sensitivity) ──
    print(f"\n  2SLS sensitivity — instrument subsets:")
    subsets = [
        (['distancefromeq'],                               "dist_eq only"),
        (['distancefromeq','fr_trade'],                    "dist + FR trade"),
        (['distancefromeq','english_frac','we_lang_frac'], "dist + language"),
        (INSTRUMENTS,                                      "all 4 instruments"),
    ]
    print(f"  {'Instruments':<35} {'β̂ (S)':>8}  {'SE':>8}  {'F-stat':>8}  {'Overid p':>9}")
    print(f"  {'-'*75}")
    for inst_list, desc in subsets:
        Z_sub = s[inst_list].values
        r = tsls(y, S, X1, Z_sub)
        se_s = np.sqrt(r['V'][1, 1])
        oid  = f"{r['overid_p']:.3f}" if not np.isnan(r['overid_p']) else "  n/a"
        print(f"  {desc:<35} {r['beta'][1]:>8.3f}  {se_s:>8.4f}  {r['F_first']:>8.2f}  {oid:>9}")

    return res_ols, res_iv, s

print()
ols_A, iv_A, sA_reg = run_regressions(sA, "Sample A — complete cases")
ols_B, iv_B, sB_reg = run_regressions(sB, "Sample B — with imputation")

# ── Compare OLS vs IV coefficients explicitly ──
print("\n" + "=" * 65)
print("  OLS vs 2SLS comparison (Social infrastructure coefficient β)")
print("=" * 65)
print(f"\n  {'Sample':<30} {'OLS β':>8}  {'2SLS β':>8}  {'Ratio IV/OLS':>13}")
print(f"  {'-'*65}")
for label, r_ols, r_iv in [("A — complete cases", ols_A, iv_A),
                             ("B — with imputation", ols_B, iv_B)]:
    b_ols = r_ols['beta'][1]
    b_iv  = r_iv['beta'][1]
    print(f"  {label:<30} {b_ols:>8.3f}  {b_iv:>8.3f}  {b_iv/b_ols:>13.2f}x")
print()
print("  Note: IV > OLS indicates measurement error dominates simultaneity bias,")
print("  consistent with Hall & Jones (1999) finding (OLS=3.29, 2SLS=5.14).")




  TABLE II — OLS and 2SLS Regressions
  Dependent variable: log(Y/L)
  Key regressor: Social infrastructure S ∈ [0,1]
  Robust (HC1) standard errors in parentheses


  [Sample A — complete cases]  N = 89

  OLS:
  Constant                   8.1611  (0.1601) ***
  Social infra (S)           3.1754  (0.2461) ***
  N                              89
  R²                         0.5994

  2SLS (instruments: dist_eq, FR trade, English frac, WE lang frac):
  Constant                   7.6115  (0.2119) ***
  Social infra (S)           4.4174  (0.4295) ***
  N                              89
  R²                         0.5077
  First-stage F               16.20
  Overid test p-val           0.089

  2SLS sensitivity — instrument subsets:
  Instruments                           β̂ (S)        SE    F-stat   Overid p
  ---------------------------------------------------------------------------
  dist_eq only                           4.935    0.6016     33.18        n/a
  dist + FR trade        

In [25]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 6 — FIGURE I: Y/L vs Social Infrastructure
# ═══════════════════════════════════════════════════════════════════════════
print("\n--- Generating Figure I ---")

# Highlight a selection of countries with labels
LABEL_ISOS = {
    'USA', 'CHE', 'NOR', 'JPN', 'GBR', 'FRA', 'DEU', 'CAN', 'AUS',
    'BRA', 'MEX', 'ARG', 'COL', 'THA', 'MYS', 'KOR',
    'CHN', 'IND', 'PAK', 'BGD', 'NGA', 'KEN', 'ETH', 'NER', 'ZAF',
    'EGY', 'GHA', 'TZA', 'UGA', 'SEN',
}

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle("Figure I — Output per Worker vs. Social Infrastructure",
             fontsize=13, fontweight='bold', y=1.01)

for ax, sample, title in [
    (axes[0], sA_reg, "Sample A: Complete cases (N={})".format(len(sA_reg))),
    (axes[1], sB_reg, "Sample B: With imputation (N={})".format(len(sB_reg))),
]:
    s = sample.dropna(subset=['log_yl', 'social_infra'])

    ax.scatter(s['social_infra'], s['log_yl'],
               color='steelblue', alpha=0.55, s=30, zorder=2, linewidths=0)

    # Fitted OLS line
    xi = np.linspace(s['social_infra'].min(), s['social_infra'].max(), 200)
    X_fit = np.column_stack([np.ones(200), xi])
    y_fit = X_fit @ ols(s['log_yl'].values,
                        np.column_stack([np.ones(len(s)), s['social_infra'].values]))['beta']
    ax.plot(xi, y_fit, color='firebrick', linewidth=1.8, zorder=3, label='OLS fit')

    # Country labels
    for _, row in s.iterrows():
        if row['iso3'] in LABEL_ISOS:
            ax.annotate(row['iso3'],
                        xy=(row['social_infra'], row['log_yl']),
                        xytext=(3, 1), textcoords='offset points',
                        fontsize=6.5, color='#333333', zorder=4)

    ax.set_xlabel("Social Infrastructure (S)", fontsize=11)
    ax.set_ylabel("log(Output per Worker)", fontsize=11)
    ax.set_title(title, fontsize=10, pad=6)
    ax.set_xlim(-0.02, 1.05)
    ax.grid(True, alpha=0.25, linewidth=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Annotate with IV coefficient
    iv_res = iv_A if sample is sA_reg else iv_B
    b  = iv_res['beta'][1]
    se = np.sqrt(iv_res['V'][1, 1])
    ax.text(0.03, 0.96, f"2SLS β = {b:.2f} ({se:.3f})",
            transform=ax.transAxes, fontsize=9,
            verticalalignment='top',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                      edgecolor='#cccccc', alpha=0.9))

plt.tight_layout()
fig1_path = OUT_DIR + 'figure_I_yl_vs_si.png'
plt.savefig(fig1_path, dpi=160, bbox_inches='tight')
plt.close()
print(f"  Saved to {fig1_path}")




--- Generating Figure I ---
  Saved to C:\Users\Adams\OneDrive\DE & E Research\outputs\figure_I_yl_vs_si.png


In [26]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 7 — FIGURE II: TFP vs Social Infrastructure
# ═══════════════════════════════════════════════════════════════════════════
print("--- Generating Figure II ---")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle("Figure II — TFP (Productivity) vs. Social Infrastructure",
             fontsize=13, fontweight='bold', y=1.01)

for ax, sample, title in [
    (axes[0], sA_enriched, "Sample A: Complete cases (N={})".format(len(sA_enriched.dropna(subset=['log_tfp','social_infra'])))),
    (axes[1], sB_enriched, "Sample B: With imputation (N={})".format(len(sB_enriched.dropna(subset=['log_tfp','social_infra'])))),
]:
    s = sample.dropna(subset=['log_tfp', 'social_infra']).copy()
    s = s[np.isfinite(s['log_tfp']) & np.isfinite(s['social_infra'])]

    ax.scatter(s['social_infra'], s['log_tfp'],
               color='darkorange', alpha=0.55, s=30, zorder=2, linewidths=0)

    # Fitted OLS line
    xi = np.linspace(s['social_infra'].min(), s['social_infra'].max(), 200)
    X_fit = np.column_stack([np.ones(200), xi])
    y_fit = X_fit @ ols(s['log_tfp'].values,
                        np.column_stack([np.ones(len(s)), s['social_infra'].values]))['beta']
    ax.plot(xi, y_fit, color='firebrick', linewidth=1.8, zorder=3)

    # Country labels
    for _, row in s.iterrows():
        if row['iso3'] in LABEL_ISOS:
            ax.annotate(row['iso3'],
                        xy=(row['social_infra'], row['log_tfp']),
                        xytext=(3, 1), textcoords='offset points',
                        fontsize=6.5, color='#333333', zorder=4)

    # Correlation
    r, p_val = stats.pearsonr(s['social_infra'], s['log_tfp'])
    ax.text(0.03, 0.96, f"r = {r:.3f}",
            transform=ax.transAxes, fontsize=9,
            verticalalignment='top',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                      edgecolor='#cccccc', alpha=0.9))

    ax.set_xlabel("Social Infrastructure (S)", fontsize=11)
    ax.set_ylabel("log(TFP)", fontsize=11)
    ax.set_title(title, fontsize=10, pad=6)
    ax.set_xlim(-0.02, 1.05)
    ax.grid(True, alpha=0.25, linewidth=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.tight_layout()
fig2_path = OUT_DIR + 'figure_II_tfp_vs_si.png'
plt.savefig(fig2_path, dpi=160, bbox_inches='tight')
plt.close()
print(f"  Saved to {fig2_path}")



--- Generating Figure II ---
  Saved to C:\Users\Adams\OneDrive\DE & E Research\outputs\figure_II_tfp_vs_si.png


In [27]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 8 — DEVIATIONS FROM ORIGINAL PAPER
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 65)
print("  DOCUMENTED DEVIATIONS FROM HALL & JONES (1999)")
print("=" * 65)
deviations = [
    ("Price base year",
     "H&J use PWT 5.6 (1985 prices). We use PWT 10.01 (2017 prices).\n"
     "    Effect: Y/L levels differ in absolute terms; relative rankings robust."),
    ("Governance data",
     "H&J use GADP (ICRG, 1986–1995 avg). We use WGI (rl+ge+cc avg, 1996).\n"
     "    Effect: Main risk is attenuation for countries with changing institutions;\n"
     "    institutional rankings highly correlated across time and source."),
    ("Openness coverage",
     "H&J use Sachs-Warner 1950–1994. Our data ends 1992 (2-year gap).\n"
     "    Effect: Negligible — affects only 2 of 40+ years in the denominator."),
    ("Mining value added",
     "H&J use 1988 values. We use 1990 (earliest available).\n"
     "    Effect: Negligible — mining shares are stable year to year."),
    ("Sample size",
     "H&J N=127 (with imputation). Our N ≈ 113–120 depending on sample.\n"
     "    Effect: Some small/transition economies missing due to data gaps."),
    ("Standard errors",
     "H&J use bootstrap (10,000 reps). We use HC1 robust standard errors.\n"
     "    Effect: SEs may differ slightly; coefficient estimates identical."),
]
for title, desc in deviations:
    print(f"\n  [{title}]")
    print(f"    {desc}")

print("\n" + "=" * 65)
print("  Analysis complete. Output files:")
print(f"    {fig1_path}")
print(f"    {fig2_path}")
print("=" * 65)


  DOCUMENTED DEVIATIONS FROM HALL & JONES (1999)

  [Price base year]
    H&J use PWT 5.6 (1985 prices). We use PWT 10.01 (2017 prices).
    Effect: Y/L levels differ in absolute terms; relative rankings robust.

  [Governance data]
    H&J use GADP (ICRG, 1986–1995 avg). We use WGI (rl+ge+cc avg, 1996).
    Effect: Main risk is attenuation for countries with changing institutions;
    institutional rankings highly correlated across time and source.

  [Openness coverage]
    H&J use Sachs-Warner 1950–1994. Our data ends 1992 (2-year gap).
    Effect: Negligible — affects only 2 of 40+ years in the denominator.

  [Mining value added]
    H&J use 1988 values. We use 1990 (earliest available).
    Effect: Negligible — mining shares are stable year to year.

  [Sample size]
    H&J N=127 (with imputation). Our N ≈ 113–120 depending on sample.
    Effect: Some small/transition economies missing due to data gaps.

  [Standard errors]
    H&J use bootstrap (10,000 reps). We use HC1 robust 